# Vickrey Auction

Vickrey auctions are a type of sealed-bid auction where all participants bid for the lot at the same time without knowing how their competitors are bidding. As a result, the winner of the auction is the bidder with the highest bid. However, the winner does not pay the highest bid, but the second-highest bid amount. This type of auction encourages the participants to bid exactly how much they value the good being sold.

https://corporatefinanceinstitute.com/resources/knowledge/other/vickrey-auction/

http://www.masfoundations.org/mas.pdf

MRobalinho : 4-12-2020


In [12]:
import numpy as np

In [1]:

#  Main start for auction vickrey
def main_auction_vickrey(matrix_arr, x_tasks, x_agents):
    print('-------------------------------------')
    print('|-- Vickrey Auction Algotithm ------|')
    print('-------------------------------------')
    # Number Agents and task
    print('Original values have Agents:', x_agents, '   Tasks:', x_tasks)

    # Transpose original Matrix
    matrix_t = matrix_arr.transpose()
    matrix_t

    arr = np.array(matrix_t)
    # Number rows and columns
    l, c = arr.shape
    
    # Create array with agent score
    score_x = []
    row, col  = 1, x_agents
    score_x = [[0 for x in range(col)] for y in range(row)] 
    score_arr = np.array(score_x)

    #max_value = max([max(l) for l in matrix_arr])
    #min_value = min([min(l) for l in matrix_arr])
    print('After transpose: Rows:',l, 'Columns:', c)

    # Creates a null list containing l(one for each task) lists, 
    # each of 4 items(4 informations), all set to 0
    # rows = tasks
    # columns (0=Task, 1=agent, 2 = bid winner)
    row, col  = l, 3;
    bk_task   = row  # backup for task number
    bk_agents = c    # Backup for agent number
    
    winners = [[0 for x in range(col)] for y in range(row)] 
    winners_arr = np.array(winners)

    # Calculate Bid Winners
    winners_arr = calc_bid(matrix_t, winners_arr)
     
    # Normalization - Some agents with many task and others with none
    verify_normalization(matrix_t, score_arr, winners_arr)
            
    #  call last scores to print
    end_run = 1
    score_arr = create_scores(winners_arr, x_agents, bk_task, end_run)
    
    # Calculate total process value
    x_total = calc_total(winners_arr)
    
    return x_total

#-----------------------------------------------------------------
# Calculate the best bids for each task
def calc_bid(matrix_t, winners_arr):
    
    # Select maximum bid in each task (each row)
    arr = np.array(matrix_t)
    
    # Number rows and columns
    l, c = arr.shape
    x_agents = c
    
    n_task = 0
    for task in matrix_t:
        max_bid = 99999   # Change to hight (99999) or Low (0)
        print('Calc winners - Task number:',n_task)
        n_agent = 0
        for agent in range(len(task)):
            
            this_bid = task[n_agent]  # Bid Value from row/column matriz              
                    
            # Select Best bid
            win = False
            if this_bid < max_bid: # Change to hight (99999) or Low (0)
                older_max_bid = max_bid
                max_bid = task[n_agent]
                print('    Win Best bid, agent ->', n_agent, '  Bid:', this_bid, ' Older Max Bid',older_max_bid)
                winners_arr[n_task,0] = n_task
                winners_arr[n_task,1] = n_agent
                winners_arr[n_task,2] = max_bid
                win = True
                
            # If  a Tie we consider the best bidder agent    
            if this_bid == max_bid and win == False:    
                                
                # Create score to obtain the best winner agent
                score_arr = create_scores(winners_arr, x_agents, bk_task, 0)
                print('    Tie agent:',n_agent, 'Task:',task[n_agent], 'Max Bid:', max_bid, ' Scores:', score_arr)
            
                ix = 0                
                # Find last winner for the same task
                if score_arr[0, n_agent] == 0: # If Agent has no task he win
                    last_bid_agent = n_agent
                    print('    ---> Best bid tie, first bid, agent ->', last_bid_agent, '  Bid:', max_bid)
                else:
                    # Find the Agent that wins more times
                    last_bid_agent = 0
                    for ix in range(len(task)):            
                        if max_bid == task[ix] and ix < n_agent:               
                            last_bid_agent = ix
                            print('    --> Calc Tie Score This:', score_arr[0, n_agent],'  Older',score_arr[0, ix]) 
                        ix += 1
                    print('    ---> Best bid tie, agent ->', last_bid_agent, '  Bid:', max_bid)
                
                winners_arr[n_task,0] = n_task
                winners_arr[n_task,1] = last_bid_agent
                winners_arr[n_task,2] = max_bid    
            n_agent += 1
        n_task += 1

    # print array winners    
    print('Winners:')
    winners_arr
     
    return winners_arr

#----------------------------------------------------------------------
def verify_normalization(matrix_t, score_arr, winners_arr):
    
    # Normalization - Some agents with many task and others with none
    try_normalization = True
    t_task, t_agents  = matrix_t.shape

    xi = 1
    print('-> Start Normalization, with :', t_task, 'tasks,  and :', t_agents,' Agents')
    print('Normaliz winners:', winners_arr)
    print('Agent scores:',score_arr)
        
    while try_normalization == True:
        # generate scores
        score_arr = create_scores(winners_arr, t_agents, bk_task, 0)
        
        n_agent_zero  = 999999
        n_agent_multi = 999999

        #print('normaliz matrix:', matrix_t)
        #print('normaliz score:',  score_arr)
        
        # Read scores to obtain One agent with zero tasks and another with may tasks      
        for line in score_arr:
            n_agent = 0
            for each_aget in line:
                #print('For agent', n_agent, line, ' Score:', line[n_agent])
                if line[n_agent] == 0:  # Search Agents without tasks, need Normalization
                    n_agent_zero = n_agent

                if line[n_agent] > 1:  # Search Agents with many tasks, need Normalization
                    n_agent_multi = n_agent            
                n_agent += 1

        print('Verify normalization, cicle:', xi,', zero agent:', n_agent_zero, ' Multi agent:', n_agent_multi )    
        if n_agent_zero != 999999 and n_agent_multi != 999999: # Has some with wulti task and some without task   
            do_normalization(matrix_t, winners_arr, n_agent_zero, n_agent_multi)
        else:
            try_normalization = False
            print('-> Finish Normalization')
        xi += 1
        
    return winners_arr, score_arr

#------------------------------------------------------
def do_normalization(matrix_t, winners_arr, agent_zero, agent_multi):

    n_task = 0
    n_do = True
    for task in winners_arr:
        max_bid = 99999   # Change to hight (99999) or Low (0)
        
        n_agent = winners_arr[n_task,1]
        #print('>> Normalization - Task number:',n_task, 'Agent:', n_agent, 'Agent zero:',agent_zero,' Agent multi:', agent_multi )
        if n_agent == agent_multi and n_do == True:
            new_bid = matrix_t[n_task, agent_zero]
            winners_arr[n_task,0] = n_task        # Dont' change the task, assign to another agent
            winners_arr[n_task,1] = agent_zero    # assign to an agent with zero task
            winners_arr[n_task,2] = new_bid       # Assign Bid from agent zero
            print('-->> Normalization - Task number:',n_task, 'Agent zero:',agent_zero,' New Bid:', new_bid )
            n_do = False  # do only one time
        n_task += 1
    return winners_arr

#------------------------------------------------------
def calc_total(winners_arr):
    print('--- Calculate totals Vickrey auction----')
 
    x_task  = 0
    x_total = 0
    for task in winners_arr:
        x_total  += winners_arr[x_task,2]
        x_task += 1
    
    xt = x_total
    print('Bid Total value:',x_total)
    print('Winners Matrix row=Tasks , Col_0 =Task number, Col_1=Agent, Col_2=Bid Value')
    winners_arr
          
    return x_total

#---------------------------------------------------------------------
# Create the agent scores
def create_scores(winners_arr, x_agents, bk_task, end_run):
    
    t_task   = bk_task
    t_agents = x_agents
    
    # Create array with agent score
    score_x = []
  
    # Transpose winners array
    winners_arr_t = winners_arr.transpose()
    winners_arr_t
    
    # Select maximum bid in each task (each row)
    n_row = 0
    # Row 1 has the winners agents, for each task
    if end_run == 1:
        print('-------------------------')
        print('WINNERS :')
 
    for row in winners_arr_t:
        if n_row == 0 :  # row with winners          
            i = 0    
            for i in row:     
                #print(' > Tasks number:',winners_arr_t[0, i], ' Winner:', winners_arr_t[1, i], '=', winners_arr_t[2, i])

                if winners_arr_t[2, i] >= 0: # Bid value need to be > 0:
                    winner_agent = winners_arr_t[1, i]  # Agent number
                    if winners_arr_t[2, i] > 0:
                        score_arr[0, winner_agent] += 1
                    if end_run == 1:
                        print('  Task > ', i, ' Agent:', winner_agent, '  Bid:', winners_arr_t[2, i])
                i += 1
        n_row += 1
        
    # Score pontuation to the agents winners
    if end_run == 1:
        print('Agent scores:',score_arr)
    return(score_arr)


In [135]:
'''
# ---- Block only for TEST ----
#
# Rows - Agents
# Columns = tasks
import numpy as np

matrix = []
matrix = [[25, 20, 21, 10, 17],
          [21, 21, 21, 11, 20]]

winners =[[ 0 , 1 ,21],
          [ 1 , 0 ,1],
          [ 2 , 0 ,1],
          [ 3 , 0 ,1],        
          [ 4 , 0 ,1]]
# Transform list to numpy array
matrix_arr = np.array(matrix)
# Transpose
x_matrix_t = matrix_arr.transpose()
#print(matrix_arr * 0.5)

x_agents = 2

#create_scores(winners_arr, x_agents, bk_task, 1)

#------- TEST AUCTION ----------------------
main_auction_vickrey(matrix_arr, bk_task, bk_agent)

#'''
'''
#----------------------------------
# TEST Normalization - Some agents with many task and others with none
#matrix_t = matrix_arr.transpose()
print('TESTE NORMALIZATION')    
score =  [[3, 1, 0, 0, 1]]
score_arr = np.array(score)
print('Scores:',score_arr)

winners =[[ 0 , 4 ,21],
          [ 6 , 0 ,1]]
winners_arr = np.array(winners)
print('Winners:')
winners_arr

# Transpose
matrix_arr = np.array(matrix)
matrix_t = matrix_arr.transpose()
print('Matrix_transposed:')
matrix_t

winners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)
#----------------------------------------

print('--- agents -----')
#print(x_matrix_t)
print('----------------')
'''

"\n#----------------------------------\n# TEST Normalization - Some agents with many task and others with none\n#matrix_t = matrix_arr.transpose()\nprint('TESTE NORMALIZATION')    \nscore =  [[3, 1, 0, 0, 1]]\nscore_arr = np.array(score)\nprint('Scores:',score_arr)\n\nwinners =[[ 0 , 4 ,21],\n          [ 6 , 0 ,1]]\nwinners_arr = np.array(winners)\nprint('Winners:')\nwinners_arr\n\n# Transpose\nmatrix_arr = np.array(matrix)\nmatrix_t = matrix_arr.transpose()\nprint('Matrix_transposed:')\nmatrix_t\n\nwinners_arr, score_arr  = verify_normalization(matrix_t, score_arr, winners_arr)\n#----------------------------------------\n\nprint('--- agents -----')\n#print(x_matrix_t)\nprint('----------------')\n"